<a href="https://colab.research.google.com/github/daothuphuong98/COVID19-NER/blob/main/BiLSTM_CRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/VinAIResearch/PhoNER_COVID19.git

Cloning into 'PhoNER_COVID19'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 58 (delta 23), reused 41 (delta 18), pack-reused 0
Receiving objects: 100% (58/58), 3.61 MiB | 8.93 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [ ]:
!pip install seqeval tensorflow_addons gensim==4.1.2 git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-flnp0xi0
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-flnp0xi0
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 864.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 42.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-4.1.2-cp310-cp310-linux_x86_64.whl size=25996772 sha256=751e253b2c409c018ffe880a6fdb618d3b079304b3fa015695dff4ef2b65185e
  Stored in directory: /root/.cache/pip/wheels/13/35/4e/dca2954de21981d0a137ff930239f0767403a617e32f19f04f
  Created wheel fo

In [ ]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from tensorflow_addons.text import crf_log_likelihood, crf_decode
from tensorflow.keras.preprocessing.text import Tokenizer, tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, TimeDistributed, Input, BatchNormalization, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers, Sequential
from seqeval.metrics import f1_score, classification_report
from keras_contrib.layers import CRF
import tensorflow.keras.optimizers as Optimizer
import plotly.express as px
from gensim.models import FastText
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
<ipython-input-3-69eaf9fad717>:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


#Data

In [ ]:
def handle_file(link):
    with open(link) as f:
        contents = f.readlines()
    idx_sentence, idx_arr, word, tag = 0, [], [], []
    for line in contents:
        arr_info = line.split(' ')
        # print(arr_info)
        if len(arr_info) == 1:
            idx_sentence += 1
            continue
        word.append(arr_info[0])
        tag.append(arr_info[1].replace('\n', ''))
        idx_arr.append(idx_sentence)
    return pd.DataFrame(
        data = {
            'idx': idx_arr,
            'word': word,
            'tag': tag
        }
    )

In [ ]:
word_train_data = handle_file('/content/PhoNER_COVID19/data/word/train_word.conll')
word_test_data = handle_file('/content/PhoNER_COVID19/data/word/test_word.conll')
word_val_data = handle_file('/content/PhoNER_COVID19/data/word/dev_word.conll')

word_train_data['word_process'] = word_train_data['word'].apply(lambda x: re.sub('[0-9]+', "<NUM>", x).lower())
word_test_data['word_process'] = word_test_data['word'].apply(lambda x: re.sub('[0-9]+', "<NUM>", x).lower())
word_val_data['word_process'] = word_val_data['word'].apply(lambda x: re.sub('[0-9]+', "<NUM>", x).lower())

filter_string = '?!"/\{}().,'
word_train_data = word_train_data[word_train_data['word_process'].apply(lambda x: x not in filter_string)]
word_test_data = word_test_data[word_test_data['word_process'].apply(lambda x: x not in filter_string)]
word_val_data = word_val_data[word_val_data['word_process'].apply(lambda x: x not in filter_string)]

word_train_data_group = word_train_data.groupby('idx').agg({'word_process': ' '.join, 'word': ' '.join, 'tag': list})
word_test_data_group = word_test_data.groupby('idx').agg({'word_process': ' '.join, 'word': ' '.join, 'tag': list})
word_val_data_group = word_val_data.groupby('idx').agg({'word_process': ' '.join, 'word': ' '.join, 'tag': list})

word_train_data_group['len_tag'] = word_train_data_group['tag'].apply(lambda x: len(x))
word_val_data_group['len_tag'] = word_val_data_group['tag'].apply(lambda x: len(x))
word_test_data_group['len_tag'] = word_test_data_group['tag'].apply(lambda x: len(x))

# train_data_group = train_data_group[train_data_group['len_tag'] <= 55].reset_index()
# val_data_group = val_data_group[val_data_group['len_tag'] <= 55].reset_index()
# test_data_group = test_data_group[test_data_group['len_tag'] <= 55].reset_index()

In [ ]:
word_train_data_group[word_train_data_group['len_tag'] < 15].sample(20)

,word_process,word,tag,len_tag
idx,,,,
3752,bệnh_nhân <num> nữ <num> tuổi trú thị_trấn đông_phú huyện quế_sơn làm nghề buôn_bán,Bệnh_nhân 622 nữ 39 tuổi trú thị_trấn Đông_Phú huyện Quế_Sơn làm nghề buôn_bán,"[O, B-PATIENT_ID, B-GENDER, B-AGE, O, O, B-LOCATION, I-LOCATION, B-LOCATION, I-LOCATION, O, O, B-JOB]",13
4544,đây là em bé thứ ba dưới một tuổi tại việt_nam nhiễm ncov,Đây là em bé thứ ba dưới một tuổi tại Việt_Nam nhiễm nCoV,"[O, O, O, O, O, O, O, O, O, O, B-LOCATION, O, O]",13
4713,hai bệnh_nhân <num> <num> đi nhiều nơi gặp nhiều người trên địa_bàn tỉnh thừa_thiên_huế,Hai bệnh_nhân 684 749 đi nhiều nơi gặp nhiều người trên địa_bàn tỉnh Thừa_Thiên_Huế,"[O, O, B-PATIENT_ID, B-PATIENT_ID, O, O, O, O, O, O, O, O, B-LOCATION, I-LOCATION]",14
5018,người ra viện cuối_cùng ở đà_nẵng là bệnh_nhân <num>,Người ra viện cuối_cùng ở Đà_Nẵng là bệnh_nhân 936,"[O, O, O, O, O, B-LOCATION, O, O, B-PATIENT_ID]",9
130,ngày <num>/<num> bệnh_nhân đến trung_tâm y_tế huyện quế_sơn khám chụp phim điều_trị ngoại_trú,Ngày 28/7 bệnh_nhân đến trung_tâm y_tế huyện Quế_Sơn khám chụp phim điều_trị ngoại_trú,"[O, B-DATE, O, O, B-LOCATION, I-LOCATION, I-LOCATION, I-LOCATION, O, O, O, O, O]",13
3261,ngày <num>/<num> mẫu bệnh_phẩm được viện vệ_sinh dịch_tễ trung_ương xét_nghiệm khẳng_định,Ngày 24/3 mẫu bệnh_phẩm được Viện Vệ_sinh Dịch_tễ Trung_ương xét_nghiệm khẳng_định,"[O, B-DATE, O, O, O, B-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATION, O, O]",11
4587,kết_quả xét_nghiệm ban_đầu có <num> mẫu âm_tính và nghi_ngờ dương_tính <num> mẫu,Kết_quả xét_nghiệm ban_đầu có 11 mẫu âm_tính và nghi_ngờ dương_tính 1 mẫu,"[O, O, O, O, O, O, O, O, O, O, O, O]",12
887,ngày <num> - <num> bà khởi_phát bệnh,Ngày 2 - 4 bà khởi_phát bệnh,"[O, B-DATE, I-DATE, I-DATE, O, O, O]",7
5026,ngày <num>/<num> anh được cách_ly tập_trung ngày <num>/<num> lấy mẫu xét_nghiệm kết_quả nghi nhiễm,Ngày 12/8 anh được cách_ly tập_trung ngày 20/8 lấy mẫu xét_nghiệm kết_quả nghi nhiễm,"[O, B-DATE, O, O, O, O, O, B-DATE, O, O, O, O, O, O]",14


In [ ]:
count_B_AGE = word_test_data_group['tag'].apply(lambda x: x.count('B-AGE')).sum()
print(count_B_AGE)

582


#CRF

In [ ]:
class CRF(L.Layer):
    def __init__(self,
                 output_dim,
                 sparse_target=True,
                 **kwargs):
        """
        Args:
            output_dim (int): the number of labels to tag each temporal input.
            sparse_target (bool): whether the the ground-truth label represented in one-hot.
        Input shape:
            (batch_size, sentence length, output_dim)
        Output shape:
            (batch_size, sentence length, output_dim)
        """
        super(CRF, self).__init__(**kwargs)
        self.output_dim = int(output_dim)
        self.sparse_target = sparse_target
        self.input_spec = L.InputSpec(min_ndim=3)
        self.supports_masking = False
        self.sequence_lengths = None
        self.transitions = None

    def build(self, input_shape):
        assert len(input_shape) == 3
        f_shape = tf.TensorShape(input_shape)
        input_spec = L.InputSpec(min_ndim=3, axes={-1: f_shape[-1]})

        if f_shape[-1] is None:
            raise ValueError('The last dimension of the inputs to `CRF` '
                             'should be defined. Found `None`.')
        if f_shape[-1] != self.output_dim:
            raise ValueError('The last dimension of the input shape must be equal to output'
                             ' shape. Use a linear layer if needed.')
        self.input_spec = input_spec
        self.transitions = self.add_weight(name='transitions',
                                           shape=[self.output_dim, self.output_dim],
                                           initializer='glorot_uniform',
                                           trainable=True)
        self.built = True

    def compute_mask(self, inputs, mask=None):
        # Just pass the received mask from previous layer, to the next layer or
        # manipulate it if this layer changes the shape of the input
        return mask

    def call(self, inputs, sequence_lengths=None, training=None, **kwargs):
        sequences = tf.convert_to_tensor(inputs, dtype=self.dtype)
        if sequence_lengths is not None:
            assert len(sequence_lengths.shape) == 2
            assert tf.convert_to_tensor(sequence_lengths).dtype == 'int32'
            seq_len_shape = tf.convert_to_tensor(sequence_lengths).get_shape().as_list()
            assert seq_len_shape[1] == 1
            self.sequence_lengths = K.flatten(sequence_lengths)
        else:
            self.sequence_lengths = tf.ones(tf.shape(inputs)[0], dtype=tf.int32) * (
                tf.shape(inputs)[1]
            )

        viterbi_sequence, _ = crf_decode(sequences,
                                         self.transitions,
                                         self.sequence_lengths)
        output = K.one_hot(viterbi_sequence, self.output_dim)
        return K.in_train_phase(sequences, output)

    @property
    def loss(self):
        def crf_loss(y_true, y_pred):
            y_pred = tf.convert_to_tensor(y_pred, dtype=self.dtype)
            log_likelihood, self.transitions = crf_log_likelihood(
                y_pred,
                tf.cast(K.argmax(y_true), dtype=tf.int32) if self.sparse_target else y_true,
                self.sequence_lengths,
                transition_params=self.transitions,
            )
            return tf.reduce_mean(-log_likelihood)
        return crf_loss

    @property
    def accuracy(self):
        def viterbi_accuracy(y_true, y_pred):
            # -1e10 to avoid zero at sum(mask)
            mask = K.cast(
                K.all(K.greater(y_pred, -1e10), axis=2), K.floatx())
            shape = tf.shape(y_pred)
            sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
            y_pred, _ = crf_decode(y_pred, self.transitions, sequence_lengths)
            if self.sparse_target:
                y_true = K.argmax(y_true, 2)
            y_pred = K.cast(y_pred, 'int32')
            y_true = K.cast(y_true, 'int32')
            corrects = K.cast(K.equal(y_true, y_pred), K.floatx())
            return K.sum(corrects * mask) / K.sum(mask)
        return viterbi_accuracy

    def compute_output_shape(self, input_shape):
        tf.TensorShape(input_shape).assert_has_rank(3)
        return input_shape[:2] + (self.output_dim,)

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'sparse_target': self.sparse_target,
            'supports_masking': self.supports_masking,
            'transitions': K.eval(self.transitions)
        }
        base_config = super(CRF, self).get_config()
        return dict(base_config, **config)

# FastText

In [ ]:
word_trained = False
syll_trained = False

In [ ]:
def train_model_fasttext(train_data, size=100, window=5, min_count=3, negative=5, sg=1, alpha=0.01, epoch=50):
    train_data = [each.split() for each in train_data]
    model_fasttext = FastText(vector_size=size, window=window, min_count=min_count,
                              workers=4, sg=1, negative=negative, alpha=alpha)
    model_fasttext.build_vocab(train_data)
    model_fasttext.train(train_data, total_examples=model_fasttext.corpus_count, epochs=epoch)
    return model_fasttext

In [ ]:
if word_trained:
    word_md = FastText.load('word_model_fasttext_gensim.bin')
else:
    word_md = train_model_fasttext([*word_train_data_group['word_process'],
                                    *word_val_data_group['word_process']],
                                    epoch=50, window=5)
    word_md.save('word_model_fasttext_gensim.bin')

In [ ]:
word_tokenizer = Tokenizer(num_words=len(word_md.wv.key_to_index), lower=True, filters="", oov_token='-OOV-')
word_tokenizer.fit_on_texts([*word_train_data_group['word_process'], *word_val_data_group['word_process']])
word_index = word_tokenizer.word_index
emb_mean, emb_std = -0.5,0.5
embed_size = 100
nb_words = len(word_index) + 1
word_embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    if word in word_md.wv.key_to_index:
        word_embedding_matrix[i] = word_md.wv.get_vector(word)

In [ ]:
word_max_length = 200
word_X_train = word_tokenizer.texts_to_sequences(word_train_data_group['word_process'])
word_X_val = word_tokenizer.texts_to_sequences(word_val_data_group['word_process'])
word_X_test = word_tokenizer.texts_to_sequences(word_test_data_group['word_process'])

word_X_train = pad_sequences(word_X_train, maxlen=word_max_length, padding='post')
word_X_val = pad_sequences(word_X_val, maxlen=word_max_length, padding='post')
word_X_test = pad_sequences(word_X_test, maxlen=word_max_length, padding='post')

In [ ]:
word_tag_tokenizer = Tokenizer(filters="", lower=False)
word_tag_tokenizer.fit_on_texts(word_train_data_group['tag'].apply(lambda x: ' '.join(x)))
word_tag_index = word_tag_tokenizer.word_index
word_tag_size = len(word_tag_index)+1

word_y_train = word_tag_tokenizer.texts_to_sequences(word_train_data_group['tag'].apply(lambda x: ' '.join(x)))
word_y_val = word_tag_tokenizer.texts_to_sequences(word_val_data_group['tag'].apply(lambda x: ' '.join(x)))
word_y_test = word_tag_tokenizer.texts_to_sequences(word_test_data_group['tag'].apply(lambda x: ' '.join(x)))

word_y_train = pad_sequences(word_y_train, maxlen=word_max_length, padding='post')
word_y_val = pad_sequences(word_y_val, maxlen=word_max_length, padding='post')
word_y_test = pad_sequences(word_y_test, maxlen=word_max_length, padding='post')

In [ ]:
word_y_train = np.asarray([to_categorical(i, num_classes=word_tag_size) for i in word_y_train])
word_y_val = np.asarray([to_categorical(i, num_classes=word_tag_size) for i in word_y_val])
word_y_test = np.asarray([to_categorical(i, num_classes=word_tag_size) for i in word_y_test])

#MODEL

## Without CRF

In [ ]:
def create_model(embeddings_matrix, vocab_size, embedding_dim, max_length):
    input = Input(shape = (max_length, ), dtype='int32', name='input_text')
    x = Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                  weights=[word_embedding_matrix])(input)
    x = Bidirectional(LSTM(units=max_length, return_sequences=True,
                                recurrent_dropout=0.01))(x)
    x = TimeDistributed(Dense(128, activation='relu', kernel_initializer='he_normal'))(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.5)(x)
    output = Dense(word_tag_size, activation='softmax', kernel_initializer='he_normal')(x)
    model_final = Model(input, output)
    model_final.compile(optimizer=Optimizer.Adam(lr=0.005), loss='categorical_crossentropy',
                        metrics=['accuracy'])
    return model_final

In [ ]:
model = create_model(word_embedding_matrix, nb_words, embed_size, word_max_length)
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_text (InputLayer)     [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 100)          451700    
                                                                 
 bidirectional (Bidirection  (None, 200, 400)          481600    
 al)                                                             
                                                                 
 time_distributed (TimeDist  (None, 200, 128)          51328     
 ributed)                                                        
                                                                 
 batch_normalization (Batch  (None, 200, 128)          512       
 Normalization)                                                  
                                                             

In [ ]:
model.fit(word_X_train, word_y_train, batch_size=64, epochs=10, validation_data = (word_X_val, word_y_val), verbose=1)

Epoch 1/10
79/79 [==============================] - 146s 2s/step - loss: 0.5742 - accuracy: 0.9300 - val_loss: 0.2430 - val_accuracy: 0.9664
Epoch 2/10
79/79 [==============================] - 130s 2s/step - loss: 0.1142 - accuracy: 0.9785 - val_loss: 0.2025 - val_accuracy: 0.9776
Epoch 3/10
79/79 [==============================] - 124s 2s/step - loss: 0.0588 - accuracy: 0.9866 - val_loss: 0.1341 - val_accuracy: 0.9875
Epoch 4/10
79/79 [==============================] - 125s 2s/step - loss: 0.0376 - accuracy: 0.9907 - val_loss: 0.1372 - val_accuracy: 0.9890
Epoch 5/10
79/79 [==============================] - 125s 2s/step - loss: 0.0275 - accuracy: 0.9931 - val_loss: 0.0619 - val_accuracy: 0.9918
Epoch 6/10
79/79 [==============================] - 122s 2s/step - loss: 0.0221 - accuracy: 0.9942 - val_loss: 0.0416 - val_accuracy: 0.9928
Epoch 7/10
79/79 [==============================] - 123s 2s/step - loss: 0.0182 - accuracy: 0.9952 - val_loss: 0.0528 - val_accuracy: 0.9935
Epoch 8/10
79

In [ ]:
def get_tags(sequences, tag_index):
    sequence_tags = []
    for sequence in sequences:
        sequence_tag = []
        for categorical in sequence:
            sequence_tag.append(tag_index.get(np.argmax(categorical)))
        sequence_tags.append(sequence_tag)
    return sequence_tags

def predict(model, tag_tokenizer, sent):
    tag_index = tag_tokenizer.word_index
    tag_size = len(tag_index) + 1
    pred = model.predict(sent)
    sequence_tags = get_tags(pred, {i: t for t, i in tag_index.items()})
    for idx, each in enumerate(sequence_tags):
        try:
           idx_cut = each.index(None)
        except:
           idx_cut = len(each) + 1
        sequence_tags[idx] = each[:idx_cut]
    return sequence_tags

In [ ]:
res = predict(model, word_tag_tokenizer, word_X_test)
print(classification_report(word_test_data_group['tag'].apply(lambda x: x[:word_max_length]), res))

94/94 [==============================] - 11s 111ms/step
                     precision    recall  f1-score   support

                AGE       0.90      0.88      0.89       582
               DATE       0.94      0.95      0.94      1654
             GENDER       0.95      0.85      0.90       462
                JOB       0.53      0.24      0.33       173
           LOCATION       0.80      0.76      0.78      4441
               NAME       0.93      0.43      0.59       318
       ORGANIZATION       0.65      0.65      0.65       771
         PATIENT_ID       0.97      0.94      0.95      2005
SYMPTOM_AND_DISEASE       0.70      0.61      0.65      1136
     TRANSPORTATION       0.92      0.79      0.85       193

          micro avg       0.84      0.79      0.81     11735
          macro avg       0.83      0.71      0.75     11735
       weighted avg       0.84      0.79      0.81     11735



## With CRF

In [ ]:
def create_model_crf(embeddings_matrix, vocab_size, embedding_dim, max_length):
    input = Input(shape = (max_length, ), dtype='int32', name='input_text')
    x = Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                  weights=[word_embedding_matrix])(input)
    x = Bidirectional(LSTM(units=max_length, return_sequences=True,
                                recurrent_dropout=0.01))(x)
    x = TimeDistributed(Dense(128, activation='relu', kernel_initializer='he_normal'))(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(word_tag_size, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.1)(x)
    crf = CRF(word_tag_size)
    output = crf(x)
    model_final = Model(input, output)
    model_final.compile(optimizer=Optimizer.Adam(lr=0.005), loss=crf.loss,
                        metrics=[crf.accuracy])

    return model_final

In [ ]:
model = create_model_crf(word_embedding_matrix, nb_words, embed_size, word_max_length)
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_text (InputLayer)     [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 100)          451700    
                                                                 
 bidirectional (Bidirection  (None, 200, 400)          481600    
 al)                                                             
                                                                 
 time_distributed (TimeDist  (None, 200, 128)          51328     
 ributed)                                                        
                                                                 
 batch_normalization (Batch  (None, 200, 128)          512       
 Normalization)                                                  
                                                             

In [ ]:
model.fit(word_X_train, word_y_train, batch_size=64, epochs=10, validation_data = (word_X_val, word_y_val))

Epoch 1/10
79/79 [==============================] - 160s 2s/step - loss: 527.3000 - viterbi_accuracy: 0.4764 - val_loss: 440.0032 - val_viterbi_accuracy: 0.9408
Epoch 2/10
79/79 [==============================] - 150s 2s/step - loss: 430.5999 - viterbi_accuracy: 0.7827 - val_loss: 581.1432 - val_viterbi_accuracy: 0.0974
Epoch 3/10
79/79 [==============================] - 133s 2s/step - loss: 363.6150 - viterbi_accuracy: 0.9509 - val_loss: 552.0652 - val_viterbi_accuracy: 0.0999
Epoch 4/10
79/79 [==============================] - 146s 2s/step - loss: 303.7389 - viterbi_accuracy: 0.9799 - val_loss: 523.3336 - val_viterbi_accuracy: 0.1007
Epoch 5/10
79/79 [==============================] - 140s 2s/step - loss: 248.4680 - viterbi_accuracy: 0.9827 - val_loss: 495.6163 - val_viterbi_accuracy: 0.1018
Epoch 6/10
79/79 [==============================] - 142s 2s/step - loss: 197.8174 - viterbi_accuracy: 0.9840 - val_loss: 469.8744 - val_viterbi_accuracy: 0.0956
Epoch 7/10
79/79 [================

In [ ]:
def get_tags(sequences, tag_index):
    sequence_tags = []
    for sequence in sequences:
        sequence_tag = []
        for categorical in sequence:
            sequence_tag.append(tag_index.get(np.argmax(categorical)))
        sequence_tags.append(sequence_tag)
    return sequence_tags

def predict(model, tag_tokenizer, sent):
    tag_index = tag_tokenizer.word_index
    tag_size = len(tag_index) + 1
    pred = model.predict(sent)
    sequence_tags = get_tags(pred, {i: t for t, i in tag_index.items()})
    for idx, each in enumerate(sequence_tags):
        try:
           idx_cut = each.index(None)
        except:
           idx_cut = len(each) + 1
        sequence_tags[idx] = each[:idx_cut]
    return sequence_tags

In [ ]:
res = predict(model, word_tag_tokenizer, word_X_test)
print(classification_report(word_test_data_group['tag'].apply(lambda x: x[:word_max_length]), res))

94/94 [==============================] - 23s 250ms/step


ValueError: ignored

In [ ]:
model.save_weights("BiLSTM_CRF/model")

**evaluate**

In [ ]:
model_load = create_model_crf(word_embedding_matrix, nb_words, embed_size, word_max_length)
model_load.load_weights("/content/BiLSTM_CRF/model")

In [ ]:
text = ["Đồng_thời", ",", "bệnh_viện", "tiếp_tục", "thực_hiện", "các", "biện_pháp", "phòng_chống", "dịch_bệnh", "COVID", "-", "19", "theo", "hướng_dẫn", "của", "Bộ", "Y_tế", "."]
text

['Đồng_thời',
 ',',
 'bệnh_viện',
 'tiếp_tục',
 'thực_hiện',
 'các',
 'biện_pháp',
 'phòng_chống',
 'dịch_bệnh',
 'COVID',
 '-',
 '19',
 'theo',
 'hướng_dẫn',
 'của',
 'Bộ',
 'Y_tế',
 '.']

In [ ]:
text = word_tokenizer.texts_to_sequences(text)
text

[[463],
 [1],
 [8],
 [147],
 [254],
 [33],
 [516],
 [272],
 [391],
 [26],
 [4],
 [1],
 [73],
 [500],
 [32],
 [77],
 [29],
 [1]]

In [ ]:
res = predict(model_load, word_tag_tokenizer, text)
res

1/1 [==============================] - 0s 467ms/step


[['O'],
 ['O'],
 ['I-ORGANIZATION'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['I-ORGANIZATION'],
 ['I-ORGANIZATION'],
 ['O'],
 ['I-DATE'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['B-ORGANIZATION'],
 ['I-ORGANIZATION'],
 ['O']]